### Transformers用于微调Text Generation任务的应用步骤
Text Generation任务目标是基于给定的输入自动生成连续的文本。通常，模型根据已经提供的上下文信息，逐词或者逐句的预测接下来的单词或者句子，直到生成满足一定长度或者达到特定条件的文本。<br>
1. 预训练阶段
* Transformer模型（如GPT、T5等）首先在大规模无标签文本数据上进行预训练。预训练过程中模型通过自回归或序列到序列（Seq2Seq）的方式生成文本：
    * 自回归文本生成：给定一段文本，模型通过预测下一个单词进行训练，直到生成完整的序列。
    * 序列到序列生成：输入一个句子或文本片段，模型生成与之对应的输出，如机器翻译或摘要生成。
* 预训练目标
    * 语言模型任务：根据之前生成的词预测下一个词。
    * 条件生成任务（例如T5）：根据输入文本片段生成目标文本。
2. 微调阶段
* 数据准备：准备好针对文本生成任务的数据集。通常包括输入文本（如提示）和目标文本（如续写段落）。
    * 对于自回归模型（如GPT-2），输入和目标文本可以是同一段文本（模型生成文本的一部分）。
    * 对于Seq2Seq模型（如T5），输入和目标是成对的，如输入一个问题，输出回答。
    * 输入格式化
        * Tokenization：使用Transformer模型的分词器将文本转换为token ID，通常包含填充、截断和特殊标记（如<CLS>、<SEP>）。
        * Attention Mask：生成Mask，指示哪些token有效，哪些是填充部分（如1表示有效，0表示填充部分）。
        * Labels：对于生成任务，标签就是我们希望模型生成的目标文本的token序列。
3. 训练
* 在微调阶段，通过使用带有输入-输出对的训练数据，训练模型生成文本。训练的目标是最小化模型生成的文本序列与目标文本序列之间的差异。
    * 前向传播：输入文本经过模型后生成输出序列，预测下一个单词或生成目标文本。
    * 损失计算：使用损失函数（如交叉熵损失）来评估模型生成的文本与目标文本的差异。
    * 反向传播：通过梯度下降优化模型参数，以减少生成文本与目标文本的差异。
    * 优化器：通常使用AdamW优化器来更新模型权重。
4. 评估与生成
* 训练完成后，可以在验证集上评估模型性能，或在测试阶段生成新的文本内容。
    * 生成文本：输入一个提示或部分文本，让模型生成扩展文本。使用模型的generate方法进行文本生成。
    * 生成策略：
        * Greedy Search：每一步生成概率最高的词。
        * Beam Search：搜索多个可能的生成路径，选择得分最高的路径。
        * Sampling：随机采样生成下一个词，通常用于生成多样化文本。


### 1. 导包

In [17]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import pandas as pd
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

### 2. 加载Hugging Face数据集

wikitext-2 数据集包含经过清理的维基百科文章，删除了超短或格式化不良的句子。数据集分为多个部分，通常包括：
1. train：训练集（大量未标注的纯文本数据，大约36718行）
2. validation：验证集（包含大约3760行文本数据，用于模型的验证）
3. test：测试集（包含大约4358行的文本数据，用于最终的测试和评估）
每个部分的数据集包含的内容主要是整篇维基百科文章的文本段落，这些文本是按行排列的。<br>

特征描述：<br>
* text
    * text列包含的是经过处理的wiki百科文本数据
    * 它是一个长文本序列，包含wiki百科文章的原始文本（未经过分词和标注）
    * 每一行代表一个文本片段，可能是一个完整的句子或者段落

In [3]:
# 加载wikitext-2数据集
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# 查看数据集的结构
print(dataset)

Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 1150549.58 examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [6]:
train_df=pd.DataFrame(dataset['train'])

In [7]:
train_df.head()

,text
0,
1,= Valkyria Chronicles III = \n
2,
3,Senjō no Valkyria 3 : Unrecorded Chronicles (...
4,"The game began development in 2010 , carrying..."


In [62]:
print(train_df['text'])

0                                                         
1                           = Valkyria Chronicles III = \n
2                                                         
3         Senjō no Valkyria 3 : Unrecorded Chronicles (...
4         The game began development in 2010 , carrying...
                               ...                        
36713     Common starlings may be kept as pets or as la...
36714     The common starling 's gift for mimicry has l...
36715     Mozart had a pet common starling which could ...
36716     Common starlings are trapped for food in some...
36717                                                     
Name: text, Length: 36718, dtype: object


### 3. 加载预训练的GPT-2模型和分词器

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

### 4. 数据处理
将文本数据转换为GPT-2可以接受的输入格式，即token ID序列。对于文本生成任务，输入和目标文本可以是相同的。

数据集的labels是模型需要预测的目标，即生成的下一个单词、词元或者句子。<br>
在文本生成任务中，数据集的输入和标签是通过移位来创建的。<br>
1. 输入（input_ids）：当前的文本片段，例如 "The capital of France is".
2. 标签（labels）：输入文本片段的下一个词或词元，即模型需要预测的目标。例如，上面的例子中，标签将是下一个词 "Paris".

In [73]:
def tokenize_and_shift_labels(examples):
    """
        将每个文本样本进行分词处理，并返回一个字典
        input_ids: 文本转换为的词汇表索引，用于模型输入
        attention_mask: 标识哪些tokens是有效的（1表示有效，0表示填充）
        labels: 对应的目标词元索引，用于模型的监督学习，预测的目标
    """
    # 对文本进行分词
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    
    # 将 inputs['input_ids'] 向右移位，作为 labels
    labels = inputs['input_ids'].copy()
    
    # 将 inputs 的下一个词作为 labels
    # 例如输入是 [101, 2003, 1037, 2304]，那么 label 应该是 [2003, 1037, 2304, 102]
    inputs["labels"] = labels
    
    return inputs

# 对数据集的所有样本进行了批量处理（batched=True)
# remove_columns=['text']标识删除了原始的text列，只保留了分词后的结果
tokenized_datasets = dataset.map(tokenize_and_shift_labels, batched=True, remove_columns=["text"])

# 设置 dataset 格式为torch，同时指定只保留input_ids用于训练或者推理（模型的实际输入）
# attention_mask则用于模型的注意力机制来区分实际token和填充部分
tokenized_datasets.set_format("torch", columns=["input_ids"])


Map: 100%|██████████| 3760/3760 [00:01<00:00, 2885.30 examples/s]


In [74]:
print(tokenized_datasets)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3760
    })
})


In [64]:
# 查看分词后的数据集
print(tokenized_datasets["train"][0])

{'input_ids': tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

### 5. 构建数据加载器
为了方便训练，我们将tokenized数据集转换为PyTorch DataLoader

In [81]:
# 确保 tokenized_datasets 包含 "labels" 列
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [82]:
# 定义数据填充器，用于将不同长度的输入填充到相同长度
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 构建训练集的数据加载器
train_dataloader = DataLoader(
    tokenized_datasets["train"],  # 使用训练集
    shuffle=True,  # 随机打乱数据
    batch_size=8,  # 每个批次的大小
    collate_fn=data_collator  # 使用数据填充器
)

# 构建验证集的数据加载器
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],  # 使用验证集
    shuffle=False,  # 验证集不需要打乱
    batch_size=8,  # 每个批次的大小
    collate_fn=data_collator  # 使用数据填充器
)

In [83]:
# 查看一个批次的数据
for batch in train_dataloader:
    print(batch.keys())  # 查看 batch 中的键
    print(batch["input_ids"].shape)  # 打印 input_ids 的形状
    print(batch["attention_mask"].shape)  # 打印 attention_mask 的形状
    print(batch["labels"].shape)  # 打印 labels 的形状
    break  # 只查看第一个批次


dict_keys(['input_ids', 'attention_mask', 'labels'])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])


### 6. 设置优化器和训练参数

In [84]:
# 设置优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 7. 训练模型

跑的时间太久了 就先不跑了！

In [86]:
# 移动模型到设备（GPU或CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练模型
model.train()

for epoch in range(3):  # 训练3个epoch
    for batch in train_dataloader:
        # 将数据移动到GPU（如有）
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)  # 使用DataCollatorForSeq2Seq会自动处理labels
        
        # 模型前向传播
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch} finished")

KeyboardInterrupt: 

### 8. 生成文本

In [ ]:
model.eval()

# 输入文本提示
input_text = "The future of AI is"
inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# 生成文本
outputs = model.generate(inputs, max_length=50, num_return_sequences=1)

# 解码生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)